In [1]:
import time
import os, gc
import pyspark
import pandas as pd
from itertools import islice
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Row
from lxml import etree as ET
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk/jre"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.12.0 pyspark-shell' 
num_of_th = 512; repartition_size = num_of_th*16; chunk_size = 5000000

In [2]:
conf = SparkConf().setAppName("MyApp") \
                  .set("spark.driver.memory", "8g") \
                  .set("spark.executor.memory", "8g")


sc = SparkContext(master = "local[*]", conf=conf)

spark = SparkSession(sc)

udf_registration = pyspark.sql.udf.UDFRegistration(spark)

spark

23/04/13 20:47:04 WARN Utils: Your hostname, siyer resolves to a loopback address: 127.0.1.1; using 10.19.149.144 instead (on interface wlp166s0)
23/04/13 20:47:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/siyer/.pyenv/versions/3.10.10/envs/mmds/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/siyer/.ivy2/cache
The jars for the packages stored in: /home/siyer/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4f66025d-f809-4033-bde5-be028907472a;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.12.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.3 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 96ms :: artifacts dl 4ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.12.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| sear

23/04/13 20:47:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
file_rdd = spark.read.text("./data/dump1.xml", wholetext=False)

file_chunk = file_rdd.take(chunk_size)

In [4]:
rdd = sc.parallelize(file_chunk)

def count_in_a_partition(idx, iterator):
    count = 0
    for _ in iterator:
        count += 1
    return idx,count

count_list = rdd.mapPartitionsWithIndex(count_in_a_partition).collect()
count_list

23/04/13 20:47:35 WARN TaskSetManager: Stage 3 contains a task of very large size (48755 KiB). The maximum recommended task size is 1000 KiB.


[0,
 312320,
 1,
 312320,
 2,
 312320,
 3,
 312320,
 4,
 312320,
 5,
 313344,
 6,
 312320,
 7,
 312320,
 8,
 312320,
 9,
 312320,
 10,
 313344,
 11,
 312320,
 12,
 312320,
 13,
 312320,
 14,
 312320,
 15,
 313152]

In [5]:
c = {
    "TITLE_O": "<title",
    "TEXT_O": "<text",
    "IP_O": "<ip",
    "USER_O": "<username",
    "TITLE_C": "</title",
    "TEXT_C": "</text",
    "IP_C": "</ip",
    "USER_C": "</username",
}


schema = StructType([
    StructField("title", StringType(), True),
    StructField("text", StringType(), True),
    StructField("ip", ArrayType(StringType()), True),
    StructField("user", ArrayType(StringType()), True)
])

# Create an empty DataFrame with the specified schema

entry_blank = {
    "title": "",
    "text": "",
    "ip": [[]],
    "user": [[]]
}

entry = entry_blank

In [6]:
debug = False
firehose = False
info = True
def log(val, level):
    if level == -1 and firehose:
        print(val)
    if level == 0 and debug:
        print(val)
    if level == 1 and info:
        print(val)

In [7]:
def return_opening_tag(event):
    if c["TITLE_O"] in event:
        return 1
    elif c["TEXT_O"] in event:
        return 2
    elif c["IP_O"] in event:
        return 3
    elif c["USER_O"] in event:
        return 4
    else:
        return 0
    

def get_values(part):
    entry_array = []
    elem_set = 0
    read = True
    entry = entry_blank
    
    try:
        cont = True
        while(cont):
            log(entry, -1)
            if read:
                event = next(part).value
            match elem_set:
                case 0:
                    match return_opening_tag(event):
                        case 1:
                            log("title", 0)
                            if entry["title"] != "" and entry["text"] != "":
                                entry_array.append(entry)
                                entry = entry_blank
                                gc.collect()
                            elem_set = 1
                            read = False
                        case 2:
                            log("text", 0)
                            elem_set = 2
                            read = False
                        case 3:
                            log("ip", 0)
                            elem_set = 3
                            read = False
                        case 4:
                            log("user", 0)
                            elem_set = 4
                            read = False

                case 1:
                    read = True
                    entry["title"] += event
                    if c["TITLE_C"] in event:
                        elem_set = 0

                case 2:
                    read = True
                    entry["text"] += event
                    if c["TEXT_C"] in event:
                        elem_set = 0

                case 3:
                    read = True
                    entry["ip"][-1].append(event)
                    if c["IP_C"] in event:
                        entry["ip"].append([])
                        elem_set = 0

                case 4:
                    read = True
                    entry["user"][-1].append(event)
                    if c["USER_C"] in event:
                        entry["user"].append([])
                        elem_set = 0
    except StopIteration:
        cont = False
    log(entry_array, -1)
    return entry_array

In [8]:
udf_registration.register(name="get_values", f=get_values, returnType=ArrayType(schema))

<function __main__.get_values(part)>

In [9]:
elems = []
for idx_,count in enumerate(count_list):
    elems = sc.runJob(rdd, lambda part: get_values(part))
    log(len(elems), 1)
    log(elems[0], 1)
    gc.collect()

23/04/13 20:47:37 WARN TaskSetManager: Stage 4 contains a task of very large size (48755 KiB). The maximum recommended task size is 1000 KiB.


ERROR:root:KeyboardInterrupt while sending command.               (0 + 16) / 16]
Traceback (most recent call last):
  File "/home/siyer/.pyenv/versions/3.10.10/envs/mmds/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/siyer/.pyenv/versions/3.10.10/envs/mmds/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/siyer/.pyenv/versions/3.10.10/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [1]:
len(elems)

NameError: name 'elems' is not defined

In [ ]:
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA

lda = LDA(k=100, seed=1, optimizer="em")
lda.setMaxIter(5)

lda.getMaxIter()

lda.clear(lda.maxIter)
model = lda.fit(df)
model.setSeed(1)

model.getTopicDistributionCol()

model.isDistributed()

localModel = model.toLocal()
localModel.isDistributed()

model.vocabSize()

model.describeTopics().show()

model.topicsMatrix()